In [549]:
import leuvenmapmatching as lv
import snman
import networkx as nx
import geopandas as gpd
from leuvenmapmatching.matcher.distance import DistanceMatcher
from leuvenmapmatching.map.inmem import InMemMap
from leuvenmapmatching import visualization as mmviz
import shapely as shp
import momepy

path='C:/DATA/CLOUD STORAGE/polybox/Research/SNMan/SNMan Shared/development/'
crs=2056

source = gpd.read_file(path + 'source_network.gpkg').to_crs(crs)
target = gpd.read_file(path + 'target_network.gpkg').to_crs(crs)

source_test_path = source.iloc[1]['geometry']

#G = nx.read_shp(path + 'target_network.gpkg')
G = nx.from_pandas_edgelist(target, source='u', target='v', edge_attr='geometry')
G = G.to_undirected()

for id, data in G.edges.items():
    u, v = id
    G.nodes[u]['geometry'] = shp.geometry.Point(data['geometry'].coords[0])
    G.nodes[v]['geometry'] = shp.geometry.Point(data['geometry'].coords[-1])


#snman.export_streetgraph(G, path + 'edges.gpkg', path + 'nodes.gpkg')

In [550]:
list(G.nodes.items())

[(4, {'geometry': <shapely.geometry.point.Point at 0x20769aa4400>}),
 (1, {'geometry': <shapely.geometry.point.Point at 0x20767810ee0>}),
 (2, {'geometry': <shapely.geometry.point.Point at 0x20767810130>}),
 (3, {'geometry': <shapely.geometry.point.Point at 0x20767810bb0>}),
 (5, {'geometry': <shapely.geometry.point.Point at 0x20767810d60>}),
 (6, {'geometry': <shapely.geometry.point.Point at 0x20767810640>}),
 (7, {'geometry': <shapely.geometry.point.Point at 0x20763865640>}),
 (8, {'geometry': <shapely.geometry.point.Point at 0x2076398eb20>}),
 (9, {'geometry': <shapely.geometry.point.Point at 0x207638656a0>}),
 (10, {'geometry': <shapely.geometry.point.Point at 0x20763865610>}),
 (11, {'geometry': <shapely.geometry.point.Point at 0x20767810d00>})]

In [551]:
map_con = InMemMap("myosm", use_latlon=False)

# please note that lv works with lat, lon (reverse order)
for id, data in G.nodes.items():
    map_con.add_node(id, (data['geometry'].y, data['geometry'].x))

for id, data in G.edges.items():
    map_con.add_edge(*id)
    map_con.add_edge(*id[::-1])

# reversed coords
path = [coords[::-1] for coords in list(source_test_path.coords)]

a = lv.matcher.distance.BaseMatcher(map_con)
matcher = DistanceMatcher(map_con, obs_noise=50)

states, _ = matcher.match(path)
nodes = matcher.path_pred_onlynodes

print("States\n------")
print(states)
print("Nodes\n------")
print(nodes)
print("")
matcher.print_lattice_stats()

Searching closeby nodes with linear search, use an index and set max_dist


States
------
[(1, 2), (2, 8)]
Nodes
------
[1, 2, 8]

Stats lattice
-------------
nbr levels               : 2
nbr lattice              : 40
avg lattice[level]       : 20.0
min lattice[level]       : 20
max lattice[level]       : 20
avg obs distance         : 5.384475772864472
last logprob             : -0.014192368410111683
last length              : 2
last norm logprob        : -0.007096184205055841


In [552]:
mmviz.plot_map(map_con, matcher=matcher,
               show_labels=True, show_matching=True, show_graph=True,
               filename="my_plot.png")

(None, None)

In [553]:
target.explode()

C:\Users\lukas\AppData\Local\Temp\ipykernel_7256\2486306270.py:1: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  target.explode()


,,name,u,v,geometry
0,0,None,4,1,"LINESTRING (2682837.382 1247900.896, 2682823.3..."
1,0,None,1,2,"LINESTRING (2682823.360 1247839.048, 2682781.7..."
2,0,None,1,3,"LINESTRING (2682823.360 1247839.048, 2682769.0..."
3,0,None,1,5,"LINESTRING (2682823.360 1247839.048, 2682867.1..."
4,0,None,5,6,"LINESTRING (2682867.140 1247819.888, 2682884.6..."
5,0,None,6,7,"LINESTRING (2682884.655 1247796.631, 2682915.8..."
6,0,None,2,8,"LINESTRING (2682781.713 1247867.798, 2682762.9..."
7,0,None,7,9,"LINESTRING (2682915.823 1247784.137, 2682976.5..."
8,0,None,9,10,"LINESTRING (2682976.514 1247781.282, 2683055.5..."
9,0,None,10,11,"LINESTRING (2683055.508 1247728.081, 2683186.0..."
